# Implementation of NCSN (Noise Conditional Score Networks)

In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!which python

Sun Sep 29 09:03:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   38C    P0              38W / 184W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
try:
    from NCSN.train import train, make_dataset, cal_noise_level, langevin
    from NCSN.utils import train_set, val_set
    from NCSN.model import UNet
except:
    from train import train, make_dataset, cal_noise_level, langevin
    from utils import train_set, val_set
    from model import UNet

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
import os

train_loader = DataLoader(train_set, 256, shuffle=True, drop_last=False, pin_memory=True)
val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

def get_outdir():
    now = datetime.datetime.now()
    time_str = now.strftime("%Y%m%d_%H%M%S")
    outdir = f"NCSN/training_data/{time_str}.out"
    return outdir

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Here are the hyperparameters

epochs = 200
criterion = nn.MSELoss()
init_sigma = 1
final_sigma = 0.01
n_sigma = 10
eps = 2e-5
T = 100

model = UNet()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

train(epochs=epochs, model=model, optimizer=optimizer, criterion=criterion, train_loader=train_loader, val_loader=val_loader, sigmas=sigmas, eps=eps, T=T, outdir=get_outdir())

Epoch 0: train loss 0.8718431016232105


Epoch 1: train loss 0.7176658369125204


Epoch 2: train loss 0.6409744833378082


Epoch 3: train loss 0.5947630390207819


Epoch 4: train loss 0.54672425693654


...

Epoch 59: train loss 0.2212479574883238


Epoch 60: train loss 0.21774003334501957


Epoch 61: train loss 0.21684002901645416


Epoch 62: train loss 0.21448510396987833


Epoch 64/200:  86%|██████████████████████████████████████████       | 202/235 [00:20<00:03, 10.04it/s, Train Loss=0.215]

In [15]:
# # load a model
# model = UNet()
# model.load_state_dict(torch.load('model.pth'))
# make_dataset(model, sigmas, eps=eps, T=T)
# print("Dataset created")
# os.system("python evaluate_cgen.py")

In [14]:
model = UNet()
eval_epoch = 70
model.load_state_dict(torch.load(f"NCSN/models/{eval_epoch}.pth"))
model.to('cuda')

eps = 2e-5
clamp = False

# x = torch.rand(10, 1, 28, 28).cuda()
x = torch.randn(10, 1, 28, 28).cuda()
x = (x+1)/2
y = langevin(model, x, sigmas, eps=eps, T=T, save=True, epochs=eval_epoch, clamp=clamp)